# AWS - RDS MySQL
### Paquetes requeridos
Librería de Python necesaria para establecer conexión a una base de datos MySQL

In [1]:
# !pip install PyMySQL


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### Credenciales
Para acceder a la BD necesitaremos tres datos:
1. Endpoint/host: la dirección donde se encuentra el servidor
2. Usuario
3. Contraseña

In [35]:
import configparser

# Cargar el archivo de configuración
config = configparser.ConfigParser()
config.read('config.ini')

# Acceder a las configuraciones
username = config['Credentials']['username']
password = config['Credentials']['password']
host = config['Credentials']['host']
port = config['Credentials']['username']

In [38]:
host

'prueba-clases-1.c12yeecso35u.us-east-1.rds.amazonaws.com'

### Conexion BD

In [39]:
import pymysql

In [40]:
'''
pymysql.cursors.DictCursor para que los resultados que devuelva sean diccionarios,
por defecto devuelve tuplas. Asi podemos acceder por clave a las columnas.
'''

db = pymysql.connect(host = host,
                     user = username,
                     password = password,
                     cursorclass = pymysql.cursors.DictCursor
)

# El objeto cursor es el que ejecutará las queries y devolverá los resultados

cursor = db.cursor()

In [41]:
host

'prueba-clases-1.c12yeecso35u.us-east-1.rds.amazonaws.com'

### Version DB

In [42]:
'''
Este es el engine version de la BD de AWS
fechtone trae la primera linea de la consulta
El execute() devuelve el numero de filas a las que ha afectado la query,
en este caso, devuelve una unica fila.

Execute se guarda en la conexion pero hasta que no hacemos commit
no se ejecutan las queries
de insert de datos y esas cosas...
'''

cursor.execute('SELECT VERSION()')
version = cursor.fetchone()
print(f'MySQL version: {version}')

MySQL version: {'VERSION()': '8.0.36'}


### Creación de DB

In [43]:
# Creamos una BD. Tenemos una instancia de MySQL, pero no una BD
# Los comandos de SQL  se suelen poner en mayuscula
create_db = '''CREATE DATABASE country_database'''
cursor.execute(create_db)

# Podemos eliminar la BD
# drop_db = '''DROP DATABASE country_database'''
# cursor.execute(drop_db)

# La volvemos a crear
# cursor.execute(create_db)
# el output es el numero de filas afectadas

1

In [44]:
cursor.execute('SHOW DATABASES')
cursor.fetchall()

[{'Database': 'country_database'},
 {'Database': 'information_schema'},
 {'Database': 'mysql'},
 {'Database': 'performance_schema'},
 {'Database': 'sys'}]

### Creación de tablas

In [45]:
'''
Creamos una tabla
Esto da error porque no le hemos dicho al servidor qué BD
queremos usar

'''

# crear una tabla que se llame country

create_table = '''
CREATE TABLE country (
    id INT NOT NULL auto_increment,
    name TEXT,
    continent TEXT,
    population INT,
    gdp DOUBLE,
    primary key (id)
)
'''
cursor.execute(create_table)

OperationalError: (1046, 'No database selected')

### Seleccionar la BD

In [46]:
# Para usar la BD  recien creada

cursor.connection.commit()
use_db = ''' USE country_database'''
cursor.execute(use_db)

0

In [47]:
# checkear todas las tablas que tiene mi db
cursor.execute('SHOW TABLES')
cursor.fetchall()

()

In [ ]:

# drop_table = '''DROP TABLE country'''
# cursor.execute(drop_table)


In [48]:
'''
Creamos una tabla
Esto da error porque no le hemos dicho al servidor qué BD
queremos usar

'''

# crear una tabla que se llame country

create_table = '''
CREATE TABLE country (
    id INT NOT NULL auto_increment,
    name TEXT,
    continent TEXT,
    population INT,
    gdp DOUBLE,
    primary key (id)
)
'''
cursor.execute(create_table)

0

### Insertar datos

In [49]:
insert_data = '''
INSERT INTO country (name, continent, population, gdp)
VALUES ('Spain', 'Europe', 47, 1.74)
'''
cursor.execute(insert_data)

1

In [50]:
name = 'France'
continent = 'Europe'
population = 54
gdp = 2

In [51]:
insert_data = '''
INSERT INTO country (name, continent, population, gdp)
VALUES ('%s', '%s', '%s', '%s')
''' % (name,continent,population,gdp)

cursor.execute(insert_data)

1

### Leer datos

In [52]:
sql = '''SELECT * FROM country'''
cursor.execute(sql)

2

In [53]:
mi_lista = cursor.fetchall()
len(mi_lista)

2

In [54]:
mi_lista

[{'id': 1,
  'name': 'Spain',
  'continent': 'Europe',
  'population': 47,
  'gdp': 1.74},
 {'id': 2,
  'name': 'France',
  'continent': 'Europe',
  'population': 54,
  'gdp': 2.0}]

### Guardar los cambios
Hay que ejecutar el commit antes de cerrar la sesión de la BD para que se guarden todos los cambios

In [55]:
db.commit()

### Excepciones
Se recomienda rodear este tipo de sentencias con un try/except, ya que suele ser software productivo y tiene que saber manejar fallos

In [21]:
sql = '''SELECT * FROM countryy'''
cursor.execute(sql)
print(cursor.fetchall())
print('Proceso finalizado')

ProgrammingError: (1146, "Table 'country_database.countryy' doesn't exist")

In [22]:
try:
    sql = '''SELECT * FROM countryy'''
    cursor.execute(sql)
    print(cursor.fetchall())
except Exception as e:
    print(e)

print('Proceso finalizado')

(1146, "Table 'country_database.countryy' doesn't exist")
Proceso finalizado


### Insertar datos de un CSV

In [56]:
# Creamos el CSV
import pandas as pd

df = pd.DataFrame({
    'name': ['Germany', 'Japan', 'Argentina', 'Brasil'],
    'continent' : ['Europe', 'Asia', 'America', 'America'],
    'population': [80, 126, 44, 209],
    'gdp': [4.3, 4.9, 0.5, 1.6]
})

df.head()

,name,continent,population,gdp
0,Germany,Europe,80,4.3
1,Japan,Asia,126,4.9
2,Argentina,America,44,0.5
3,Brasil,America,209,1.6


In [57]:
df.to_csv('data/country_data.csv', sep=';')

In [58]:
# leemos los datos
df = pd.read_csv('data/country_data.csv', sep=';', index_col=0)
df.head()

,name,continent,population,gdp
0,Germany,Europe,80,4.3
1,Japan,Asia,126,4.9
2,Argentina,America,44,0.5
3,Brasil,America,209,1.6


In [ ]:
# !pip install sqlalchemy==1.4.46

In [59]:
print(username)
print(password)
print(host)
print('country_database')

admin
12345678
prueba-clases-1.c12yeecso35u.us-east-1.rds.amazonaws.com
country_database


In [60]:
from sqlalchemy import create_engine

# create sqlalchemy engine

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(user = username, pw = password, host = host, db = 'country_database'))
# engine = create_engine("mysql+pymysql://my_user:my_password@my_host/my_database")

In [61]:
# insertamos todo el dataframe
df.to_sql(name='country', con=engine, if_exists= 'append', index=False)


4

In [62]:
db.commit()

In [63]:
# leemos los datos para comprobar que se han ingestado correctamente
sql = '''SELECT * FROM country'''
cursor.execute(sql)
mi_tabla = cursor.fetchall()
mi_tabla

[{'id': 1,
  'name': 'Spain',
  'continent': 'Europe',
  'population': 47,
  'gdp': 1.74},
 {'id': 2,
  'name': 'France',
  'continent': 'Europe',
  'population': 54,
  'gdp': 2.0},
 {'id': 3,
  'name': 'Germany',
  'continent': 'Europe',
  'population': 80,
  'gdp': 4.3},
 {'id': 4,
  'name': 'Japan',
  'continent': 'Asia',
  'population': 126,
  'gdp': 4.9},
 {'id': 5,
  'name': 'Argentina',
  'continent': 'America',
  'population': 44,
  'gdp': 0.5},
 {'id': 6,
  'name': 'Brasil',
  'continent': 'America',
  'population': 209,
  'gdp': 1.6}]

In [64]:
mi_tabla[4]['population']

44

In [65]:
pd.DataFrame(mi_tabla)

,id,name,continent,population,gdp
0,1,Spain,Europe,47,1.74
1,2,France,Europe,54,2.00
2,3,Germany,Europe,80,4.30
3,4,Japan,Asia,126,4.90
4,5,Argentina,America,44,0.50
5,6,Brasil,America,209,1.60


### Cerrar la conexión cuando acabemos

In [66]:
# Cerrar conexión mysql
db.close()

In [67]:
# Cerrar conexión sqlalchuemy
engine.dispose()